# Imports & Initialization

In [ ]:
# !pip install sympy tqdm dill # uncomment if the imports don't work
import os
import dill
import sympy
from sympy import symbols, Matrix, oo, Eq, simplify, solve, latex, denom, numer, sqrt, collect, degree
from tqdm import tqdm # to create progress bars
from google.colab import files
from multiprocessing import Pool, Value, cpu_count, Manager
from pprint import pprint

# Create the directory if it doesn't exist
os.makedirs("Outputs", exist_ok=True)
print("# of cores: {}".format(os.cpu_count())) # is 96 for TPU v2-8
# symbols? #use this to find documentation on any object/functionts

#

# of cores: 96


In [27]:
# Define symbolic variables
s = symbols('s')
R1, R2, R3, R4, R5, RL, Rs = symbols('R1 R2 R3 R4 R5 RL Rs')
C1, C2, C3, C4, C5, CL = symbols('C1 C2 C3 C4 C5 CL')
L1, L2, L3, L4, L5, LL = symbols('L1 L2 L3 L4 L5 LL')

a11 = symbols('a11')
a12 = symbols('a12')
a21 = symbols('a21')
a22 = symbols('a22')

# Transmission matrix coefficients
gm, ro, Cgd, Cgs = symbols('gm ro Cgd Cgs')
T_select ={
"simple" : Matrix([[0, -1/gm],[0, 0]]),
"symbolic" : Matrix([[a11, a12],[a21, a22]]),
"some_parasitic" : Matrix([[-1/(gm*ro), -1/gm],[0, 0]]),
"full_parasitic" : Matrix([[(1/ro + s*Cgd)/(s*Cgd - gm), 1/(s*Cgd - gm)],[(Cgd*Cgs*ro*s + Cgd*gm*ro + Cgs + Cgd)*s/(s*Cgd - gm), (Cgs+Cgd)*s/(s*Cgd - gm)]])
}
# Select a transimision matrix
T_a = T_select["symbolic"]
T_b = T_select["symbolic"]

# Define impedance arrays
Zzs = [Rs, 1/(s*C1), s*L1]
Zz1 = [R1, 1/(s*C1), R1/(1 + R1*C1*s), R1 + 1/(C1*s), s*L1 + 1/(s*C1)]
Zz2 = [R2, 1/(s*C2), R2/(1 + R2*C2*s), R2 + 1/(C2*s), s*L2 + 1/(s*C2)]
Zz3 = [R3, 1/(s*C3), s*L3, R3/(1 + R3*C3*s), (s*L3 + 1/(s*C3)), (L3*s)/(1 + L3*C3*s**2)]
Zz4 = [R4, 1/(s*C4), s*L4, R4/(1 + R4*C4*s), (s*L4 + 1/(s*C4)), (L4*s)/(1 + L4*C4*s**2)]
Zz5 = [R5, 1/(s*C5), s*L5, R5/(1 + R5*C5*s), (s*L5 + 1/(s*C5)), (L5*s)/(1 + L5*C5*s**2)]
ZzL = [RL, 1/(s*CL), s*LL, RL/(1 + RL*CL*s), (LL*s)/(1 + LL*CL*s**2)]

m = len(Zzs)
n = len(Zz1)
o = len(Zz2)
p = len(Zz3)
q = len(Zz4)
r = len(Zz5)
st = len(ZzL)

# Combine all impedances
impedance_combinations = []
for zs in Zzs:
    for z1 in Zz1:
        for z2 in Zz2:
            for z3 in Zz3:
                for z4 in Zz4:
                    for z5 in Zz5:
                        for zl in ZzL:
                            impedance_combinations.append([zs, z1, z2, z3, z4, z5, zl])

# Algorithms Definitions

## Algorithm 1: Attempt to Parallelize

In [ ]:
def chunk_data(data, num_chunks):
    """Split data into approximately equal-sized chunks."""
    chunk_size = len(data) // num_chunks
    return [data[i:i + chunk_size] for i in range(0, len(data), chunk_size)]

def process_combinations(batch, progress_counter):
    """Process a batch of impedance combinations and update progress."""
    results = []
    for combo in batch:
        Zs, Z1, Z2, Z3, Z4, Z5, ZL = combo

        # Define the variables and equations
        Vip, Va, Von, Vx, Vop, Vb, Vin, I1a, I1b, I2a, I2b = symbols('Vip Va Von Vx Vop Vb Vin I1a I1b I2a I2b')
        V2a, V2b, V1a, V1b = symbols('V2a V2b V1a V1b')

        # Define nodal equations (Eq. 3a-3i) -> list[ Eq(left-hand-side, right-hand-side), ... ]
        equations = [
          # 3a
          Eq(0, I1a + I2a + I1b + I2b + (Vb-Vx)/Z1 + (Von - Vx)/Z2 + (Vop - Vx)/Z2 + (Va - Vx)/Z1),
          # 3b
          Eq(I1a, (Vip - Va)/Zs + (Von - Va)/Z3 + (Vx - Va)/Z1 + (Vop - Va)/Z5),
          # 3c
          Eq(I1b, (Vin - Vb)/Zs + (Vop - Vb)/Z3 + (Vx - Vb)/Z1 + (Von - Vb)/Z5),
          # 3d
          Eq(I2a, (Va - Von)/Z3 + (Vx - Von)/Z2 + (0 - Von)/ZL + (Vop - Von)/Z4 + (Vb - Von)/Z5),
          # 3e
          Eq(I2b, (Vb - Vop)/Z3 + (Vx - Vop)/Z2 + (0 - Vop)/ZL + (Von - Vop)/Z4 + (Va - Vop)/Z5),
          # 3f
          Eq(Vx + V2a, Vop),
          Eq(Vx + V2b, Von),
          # 3g
          Eq(Vx + V1a, Va),
          Eq(Vx + V1b, Vb),
          # 3h: Transistor a
          Eq(V1a, T_a[0,0]*V2a - T_a[0,1]*I2a),
          Eq(I1a, T_a[1,0]*V2a + T_a[1,1]*I2a),
          # 3i: Transistor b
          Eq(V1b, T_b[0,0]*V2b - T_b[0,1]*I2b),
          Eq(I1b, T_b[1,0]*V2b + T_b[1,1]*I2b)
        ]

        solution = solve(equations, [Vop, Von, I1a, I2a, V2a, V1a, V1b, V2b, I1b, I2b, Vip, Vin, Vx])

        if solution:
            Hs = simplify((solution[Vop] - solution[Von]) / (solution[Vip] - solution[Vin]))
            results.append(Hs)

        # Increment the shared counter
        with progress_counter.get_lock():
            progress_counter.value += 1

    return results

def run_in_parallel(impedance_combinations):
    num_workers = cpu_count()
    chunks = chunk_data(impedance_combinations, num_workers)

    # Use a Manager to create a shared counter
    with Manager() as manager:
        progress_counter = manager.Value('i', 0)

        # Create a tqdm progress bar
        with tqdm(total=len(impedance_combinations), desc="Overall progress") as pbar:
            with Pool(processes=num_workers) as pool:
                # Wrap process_combinations to include the shared counter
                results = pool.starmap(
                    process_combinations,
                    [(chunk, progress_counter) for chunk in chunks]
                )

                # Continuously update the progress bar in the main process
                while progress_counter.value < len(impedance_combinations):
                    pbar.n = progress_counter.value
                    pbar.refresh()

    return [hs for batch in results for hs in batch]

# run_in_parallel(impedance_combinations)

## Algorithm 1: No parallelization
Loop through all impedance combinations and find H(s)

In [28]:
def getTFs(batch):
  numOfSolutions = 0
  counter = 0
  TFs = []
  for combo in tqdm(batch, desc="Getting the TFs", unit="combo"):
      Zs, Z1, Z2, Z3, Z4, Z5, ZL = combo

      Vip, Va, Von, Vx, Vop, Vb, Vin, I1a, I1b, I2a, I2b = symbols('Vip Va Von Vx Vop Vb Vin I1a I1b I2a I2b')
      V2a, V2b, V1a, V1b =  symbols('V2a V2b V1a V1b')

      # Define nodal equations (Eq. 3a-3i) -> list[ Eq(left-hand-side, right-hand-side), ... ]
      equations = [
          # 3a
          Eq(0, I1a + I2a + I1b + I2b + (Vb-Vx)/Z1 + (Von - Vx)/Z2 + (Vop - Vx)/Z2 + (Va - Vx)/Z1),
          # 3b
          Eq(I1a, (Vip - Va)/Zs + (Von - Va)/Z3 + (Vx - Va)/Z1 + (Vop - Va)/Z5),
          # 3c
          Eq(I1b, (Vin - Vb)/Zs + (Vop - Vb)/Z3 + (Vx - Vb)/Z1 + (Von - Vb)/Z5),
          # 3d
          Eq(I2a, (Va - Von)/Z3 + (Vx - Von)/Z2 + (0 - Von)/ZL + (Vop - Von)/Z4 + (Vb - Von)/Z5),
          # 3e
          Eq(I2b, (Vb - Vop)/Z3 + (Vx - Vop)/Z2 + (0 - Vop)/ZL + (Von - Vop)/Z4 + (Va - Vop)/Z5),
          # 3f
          Eq(Vx + V2a, Vop),
          Eq(Vx + V2b, Von),
          # 3g
          Eq(Vx + V1a, Va),
          Eq(Vx + V1b, Vb),
          # 3h: Transistor a
          Eq(V1a, T_a[0,0]*V2a - T_a[0,1]*I2a),
          Eq(I1a, T_a[1,0]*V2a + T_a[1,1]*I2a),
          # 3i: Transistor b
          Eq(V1b, T_b[0,0]*V2b - T_b[0,1]*I2b),
          Eq(I1b, T_b[1,0]*V2b + T_b[1,1]*I2b)
          ]

      # Solve for transfer function
      solution = solve(equations, [
          Vop, Von, I1a, I2a, V2a, V1a, V1b, V2b,
          I1b, I2b, Vip, Vin, Vx
      ])

      if solution:
          # Compute transfer function and append to list
          numOfSolutions += 1
          Hs = (solution[Vop] - solution[Von]) / (solution[Vip] - solution[Vin])
          TFs.append(simplify(Hs))

      counter += 1
      if counter % 25 == 0:
          print("")
          print("Processed {} combinations".format(counter))
          print("Number of solutions found: {}".format(numOfSolutions))

  return TFs

## Algorithm 2: Computing w0, Q, Qzz, K_LP, K_HP, K_BP
IN -> H(s), out -> temp = 0 if valid values (else temp = 1, meaning all values are invalid since filter total order is out of bound)

In [29]:
def _computeFilterParameters(tf): # HELPER FUNCTION
    s = symbols('s')
    temp = 0

    # Determine numerator and denominator of H(s)
    dn = denom(tf)  # Denominator
    nm = numer(tf)  # Numerator
    DenOrder = degree(dn, s)
    NumOrder = degree(nm, s)

    # Coefficients of denominator
    a = dn.coeff(s, 2)  # Coefficient of s^2
    b = dn.coeff(s, 1)  # Coefficient of s^1
    c = dn.coeff(s, 0)  # Coefficient of s^0

    # Validate filter order and coefficients
    if a == 0 or b == 0 or c == 0 or DenOrder > 2 or NumOrder > DenOrder:
        temp = 1
        return temp, {'message': "Invalid "}

    # Compute Q, w, Bandwidth
    Q = simplify((a / b) * sqrt(c / a))
    wo_sqr = simplify(c / a)
    wo = sqrt(wo_sqr)
    Bandwidth = wo / Q

    # Coefficients of numerator
    bhp = nm.coeff(s, 2)  # Coefficient of s^2
    bbp = nm.coeff(s, 1)  # Coefficient of s^1
    blp = nm.coeff(s, 0)  # Coefficient of s^0

    # Calculate filter constants
    K_LP = simplify(blp / (a * wo**2))
    K_HP = simplify(bhp / a)
    K_BP = simplify(bbp / (a * Bandwidth))

    # Compute Qz if conditions are met
    Qz = None
    if K_LP == K_HP and K_BP != 0 and K_LP != 0:
        Qz = simplify(K_LP * Q / K_BP)

    # Return results
    return temp, {
        "Q": Q,
        "wo": wo,
        "Bandwidth": Bandwidth,
        "K_LP": K_LP,
        "K_HP": K_HP,
        "K_BP": K_BP,
        "Qz": Qz,
    }

def computeFilterParameters(TFs, impedanceCombinations):
    """
    Compute filter parameters for a list of transfer functions and impedance combinations.

    Parameters:
    - TFs: List of transfer functions.
    - impedanceCombinations: List of corresponding impedance combinations.

    Returns:
    - List of dictionaries containing impedance combination, transfer function, and parameters.
    """
    output = []
    # Wrap the zip iterator with tqdm for progress tracking
    for tf, impedanceCombo in tqdm(zip(TFs, impedanceCombinations),
                                    total=len(TFs),
                                    desc="Computing filter parameters",
                                    unit="filter"):
        temp, parameters = _computeFilterParameters(tf)
        if temp == 0:
            output.append({"Impedance": impedanceCombo, "TF": tf, "Parameters": parameters})
        else:
            output.append({"Impedance": impedanceCombo, "TF": tf, "Parameters": None})
    return output



## Algorithm 3: Determine Filter Type
* IN -> Parameters from Algorithm 2: w0, Q, Qz, K_LP, K_HP, K_BP
* OUT -> Filter type classification (e.g., HP, BP, LP, BS, GE)





In [30]:
def getFilterType(parameters):
    """
    Determine the type of filter based on K_LP, K_HP, K_BP, and Qz.

    Parameters:
    - parameters: dict containing Q, wo, K_LP, K_HP, K_BP, and Qz.

    Returns:
    - List of filter types and associated parameters.
    """
    temp = 0  # Use as a validity flag
    K_LP = parameters.get("K_LP", 0)
    K_HP = parameters.get("K_HP", 0)
    K_BP = parameters.get("K_BP", 0)
    Q = parameters.get("Q")
    wo = parameters.get("wo")
    Qz = parameters.get("Qz")

    # Result containers
    filter_results = {
        "High-Pass": [],
        "Band-Pass": [],
        "Low-Pass": [],
        "Band-Stop": [],
        "Gain Equalizer": []
    }

    # Classification based on filter constants
    if temp == 0 and K_BP == 0 and K_LP == 0:
        filter_results["High-Pass"].append({"wo": wo, "Q": Q, "K_HP": K_HP})

    if temp == 0 and K_HP == 0 and K_LP == 0:
        filter_results["Band-Pass"].append({"wo": wo, "Q": Q, "K_BP": K_BP})

    if temp == 0 and K_HP == 0 and K_BP == 0:
        filter_results["Low-Pass"].append({"wo": wo, "Q": Q, "K_LP": K_LP})

    if temp == 0 and K_BP == 0 and K_HP != 0 and K_LP != 0:
        filter_results["Band-Stop"].append({"wo": wo, "Q": Q, "K_HP": K_HP, "K_LP": K_LP})

    if temp == 0 and K_LP == K_HP and K_BP != 0 and K_LP != 0:
        if Qz is None:
            Qz = simplify(K_LP * Q / K_BP)
        filter_results["Gain Equalizer"].append({"wo": wo, "Q": Q, "Qz": Qz, "K_LP": K_LP})

    return filter_results

def classifyFilters(tf_results):
    """
    Classify transfer functions into filter types.

    Parameters:
    - tf_results: List of transfer function results from `computeFilterParameters`.

    Returns:
    - Filter classifications.
    """
    classifiedResults = []
    for result in tf_results:
        parameters = result.get("Parameters")
        if parameters:
            filter_types = getFilterType(parameters)
            classifiedResults.append({
                "Impedance": result["Impedance"],
                "TF": result["TF"],
                "Filter Types": filter_types
            })
        else:
            classifiedResults.append({
                "Impedance": result["Impedance"],
                "TF": result["TF"],
                "Filter Types": None
            })
    return classifiedResults

## Testing all algorithms

In [6]:
# Testing Algorithm 1
impedanceBatch = impedance_combinations[0:10]
TFs = getTFs(impedanceBatch)

# Output summary of results
print("Number of transfer functions found: {}".format(len(TFs)))
for i, tf in enumerate(TFs, 1):
    print("TF {}: {}".format(i, tf))


Getting the TFs: 100%|██████████| 10/10 [00:39<00:00,  3.94s/combo]

Number of transfer functions found: 10
TF 1: R1*R2*R4*RL*(R3*R5*gm + R3 - R5)/(R1*R2*R3*R4*R5 - R1*R2*R3*R4*RL*Rs*gm + R1*R2*R3*R4*RL + R1*R2*R3*R4*Rs + 2*R1*R2*R3*R5*RL + 2*R1*R2*R3*RL*Rs + R1*R2*R4*R5*RL*Rs*gm + R1*R2*R4*R5*RL + R1*R2*R4*R5*Rs + 4*R1*R2*R4*RL*Rs + 2*R1*R2*R5*RL*Rs + R1*R3*R4*R5*RL + R1*R3*R4*RL*Rs + R1*R4*R5*RL*Rs + R2*R3*R4*R5*Rs + R2*R3*R4*RL*Rs + 2*R2*R3*R5*RL*Rs + R2*R4*R5*RL*Rs + R3*R4*R5*RL*Rs)
TF 2: R1*R2*R4*(R3*R5*gm + R3 - R5)/(CL*R1*R2*R3*R4*R5*s + CL*R1*R2*R3*R4*Rs*s + CL*R1*R2*R4*R5*Rs*s + CL*R2*R3*R4*R5*Rs*s - R1*R2*R3*R4*Rs*gm + R1*R2*R3*R4 + 2*R1*R2*R3*R5 + 2*R1*R2*R3*Rs + R1*R2*R4*R5*Rs*gm + R1*R2*R4*R5 + 4*R1*R2*R4*Rs + 2*R1*R2*R5*Rs + R1*R3*R4*R5 + R1*R3*R4*Rs + R1*R4*R5*Rs + R2*R3*R4*Rs + 2*R2*R3*R5*Rs + R2*R4*R5*Rs + R3*R4*R5*Rs)
TF 3: LL*R1*R2*R4*s*(-R3*R5*gm - R3 + R5)/(LL*R1*R2*R3*R4*Rs*gm*s - LL*R1*R2*R3*R4*s - 2*LL*R1*R2*R3*R5*s - 2*LL*R1*R2*R3*Rs*s - LL*R1*R2*R4*R5*Rs*gm*s - LL*R1*R2*R4*R5*s - 4*LL*R1*R2*R4*Rs*s - 2*LL*R1*R2*R5*Rs*s - LL*R1*

In [8]:
# Testting Algorithm 2
s = symbols('s')

Hs = (2) / (s**2 + 1*s + 3)
# print(Hs)

x = [impedance_combinations[1]]
outputs = computeFilterParameters([Hs], x)
# pprint(outputs)

filterParameters = computeFilterParameters(TFs, impedance_combinations[0:10])

Computing filter parameters: 100%|██████████| 10/10 [00:01<00:00,  6.13filter/s]


In [ ]:
pprint(classifyFilters(filterParameters))

# Experiment Runs

In [31]:
Zs, Z1, Z2, Z3, Z4, Z5, ZL = symbols("Zs Z1 Z2 Z3 Z4 Z5 ZL")
batch = [[Zs, Z1, Z2, Z3, Z4, Z5, ZL]]
TFs = getTFs(batch)
filterParameters = computeFilterParameters(TFs, batch)
classifyFilters(filterParameters)

# Output summary of results
print("Number of transfer functions found: {}".format(len(TFs)))
for i, tf in enumerate(TFs, 1):
    print("TF {}: {}".format(i, tf))

Computing filter parameters: 100%|██████████| 1/1 [00:00<00:00, 53.29filter/s]

Number of transfer functions found: 1
TF 1: Z1*Z2*Z4*ZL*(-Z3*Z5 + Z3*a12 - Z5*a12)/(-Z1*Z2*Z3*Z4*Z5*ZL*Zs*a21 - Z1*Z2*Z3*Z4*Z5*ZL*a11 - Z1*Z2*Z3*Z4*Z5*Zs*a22 + Z1*Z2*Z3*Z4*Z5*a12 + Z1*Z2*Z3*Z4*ZL*Zs*a11*a22 - Z1*Z2*Z3*Z4*ZL*Zs*a11 + Z1*Z2*Z3*Z4*ZL*Zs*a12*a21 - Z1*Z2*Z3*Z4*ZL*Zs*a22 + Z1*Z2*Z3*Z4*ZL*Zs + Z1*Z2*Z3*Z4*ZL*a12 + Z1*Z2*Z3*Z4*Zs*a12 - 2*Z1*Z2*Z3*Z5*ZL*Zs*a22 + 2*Z1*Z2*Z3*Z5*ZL*a12 + 2*Z1*Z2*Z3*ZL*Zs*a12 - Z1*Z2*Z4*Z5*ZL*Zs*a11*a22 - Z1*Z2*Z4*Z5*ZL*Zs*a11 - Z1*Z2*Z4*Z5*ZL*Zs*a12*a21 - Z1*Z2*Z4*Z5*ZL*Zs*a22 - Z1*Z2*Z4*Z5*ZL*Zs + Z1*Z2*Z4*Z5*ZL*a12 + Z1*Z2*Z4*Z5*Zs*a12 + 4*Z1*Z2*Z4*ZL*Zs*a12 + 2*Z1*Z2*Z5*ZL*Zs*a12 - Z1*Z3*Z4*Z5*ZL*Zs*a22 + Z1*Z3*Z4*Z5*ZL*a12 + Z1*Z3*Z4*ZL*Zs*a12 + Z1*Z4*Z5*ZL*Zs*a12 - Z2*Z3*Z4*Z5*ZL*Zs*a11 + Z2*Z3*Z4*Z5*Zs*a12 + Z2*Z3*Z4*ZL*Zs*a12 + 2*Z2*Z3*Z5*ZL*Zs*a12 + Z2*Z4*Z5*ZL*Zs*a12 + Z3*Z4*Z5*ZL*Zs*a12)


In [33]:
classifyFilters(filterParameters)


[{'Impedance': [Zs, Z1, Z2, Z3, Z4, Z5, ZL],
  'TF': Z1*Z2*Z4*ZL*(-Z3*Z5 + Z3*a12 - Z5*a12)/(-Z1*Z2*Z3*Z4*Z5*ZL*Zs*a21 - Z1*Z2*Z3*Z4*Z5*ZL*a11 - Z1*Z2*Z3*Z4*Z5*Zs*a22 + Z1*Z2*Z3*Z4*Z5*a12 + Z1*Z2*Z3*Z4*ZL*Zs*a11*a22 - Z1*Z2*Z3*Z4*ZL*Zs*a11 + Z1*Z2*Z3*Z4*ZL*Zs*a12*a21 - Z1*Z2*Z3*Z4*ZL*Zs*a22 + Z1*Z2*Z3*Z4*ZL*Zs + Z1*Z2*Z3*Z4*ZL*a12 + Z1*Z2*Z3*Z4*Zs*a12 - 2*Z1*Z2*Z3*Z5*ZL*Zs*a22 + 2*Z1*Z2*Z3*Z5*ZL*a12 + 2*Z1*Z2*Z3*ZL*Zs*a12 - Z1*Z2*Z4*Z5*ZL*Zs*a11*a22 - Z1*Z2*Z4*Z5*ZL*Zs*a11 - Z1*Z2*Z4*Z5*ZL*Zs*a12*a21 - Z1*Z2*Z4*Z5*ZL*Zs*a22 - Z1*Z2*Z4*Z5*ZL*Zs + Z1*Z2*Z4*Z5*ZL*a12 + Z1*Z2*Z4*Z5*Zs*a12 + 4*Z1*Z2*Z4*ZL*Zs*a12 + 2*Z1*Z2*Z5*ZL*Zs*a12 - Z1*Z3*Z4*Z5*ZL*Zs*a22 + Z1*Z3*Z4*Z5*ZL*a12 + Z1*Z3*Z4*ZL*Zs*a12 + Z1*Z4*Z5*ZL*Zs*a12 - Z2*Z3*Z4*Z5*ZL*Zs*a11 + Z2*Z3*Z4*Z5*Zs*a12 + Z2*Z3*Z4*ZL*Zs*a12 + 2*Z2*Z3*Z5*ZL*Zs*a12 + Z2*Z4*Z5*ZL*Zs*a12 + Z3*Z4*Z5*ZL*Zs*a12),
  'Filter Types': None}]

# Saving the environment parameters
Use the code block below to save the experiment.

In [32]:
# !sudo apt update
# !sudo apt install texlive-latex-base

# # Create a Latex file to report the transfer functions
header = r"""
\documentclass{article}
\usepackage{amsmath}
\usepackage{geometry}
\geometry{landscape, a1paper, margin=1in}  % Adjust paper size and margins
\begin{document}
\section*{Derived Transfer Functions}
"""

footer = r"\end{document}"

# LaTeX filename
latex_filename = "transfer_functions.tex"

# Write the LaTeX code into the file
with open(latex_filename, "w") as latex_file:
    latex_file.write(header)
    for i, tf in enumerate(TFs, 1):
        latex_file.write(f"\\subsection*{{Transfer Function {i}}}\n")
        latex_file.write(f"\\[ H(s) = {latex(tf)} \\]\n")
        # # Add a page break after each equation (optional)
        # latex_file.write("\\newpage\n")
    latex_file.write(footer)

# Compile the LaTeX file into a PDF (make sure pdflatex is installed)
!pdflatex transfer_functions.tex


# Optional: download the PDF
# files.download("transfer_functions.pdf")

This is pdfTeX, Version 3.141592653-2.6-1.40.22 (TeX Live 2022/dev/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
(./transfer_functions.tex
LaTeX2e <2021-11-15> patch level 1
L3 programming layer <2022-01-21>
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2021/10/04 v1.4n Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))
(/usr/share/texlive/texmf-dist/tex/latex/amsmath/amsmath.sty
For additional information on amsmath, use the `?' option.
(/usr/share/texlive/texmf-dist/tex/latex/amsmath/amstext.sty
(/usr/share/texlive/texmf-dist/tex/latex/amsmath/amsgen.sty))
(/usr/share/texlive/texmf-dist/tex/latex/amsmath/amsbsy.sty)
(/usr/share/texlive/texmf-dist/tex/latex/amsmath/amsopn.sty))
(/usr/share/texlive/texmf-dist/tex/latex/geometry/geometry.sty
(/usr/share/texlive/texmf-dist/tex/latex/graphics/keyval.sty)
(/usr/share/texlive/texmf-dist/tex/generic/iftex/ifvtex.sty
(/usr/shar

In [ ]:
# Saving the variable with dill
variables = {'TFs': TFs, 'Impedance_combinations': impedance_combinations}

with open('sympy_variables.dill', 'wb') as f:
    dill.dump(variables, f)

In [ ]:
# Loading the variabels with dill
with open('sympy_variables.dill', 'rb') as f:
    loaded_variables = dill.load(f)

# Access the loaded symbolic expressions and variables
TFs = loaded_variables['TFs']
Impedance_combinations = loaded_variables['Impedance_combinations']